In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "trl<0.9.0" xformers

In [2]:
import unsloth
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
max_seq_length = 4096
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen2.5-Coder-7B-Instruct",
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True,
)

==((====))==  Unsloth 2025.9.5: Fast Qwen2 patching. Transformers: 4.56.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r=32,  # Higher rank for complex coding patterns
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=32,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
)

Unsloth 2025.9.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [5]:
model.print_trainable_parameters()

trainable params: 80,740,352 || all params: 7,696,356,864 || trainable%: 1.0491


In [7]:
from datasets import load_dataset
dataset = load_dataset("iamtarun/python_code_instructions_18k_alpaca", split="train[:3000]")

README.md:   0%|          | 0.00/905 [00:00<?, ?B/s]

data/train-00000-of-00001-8b6e212f3e1ece(…):   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18612 [00:00<?, ? examples/s]

In [8]:
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'prompt'],
    num_rows: 3000
})

In [9]:
def format_code_generation(examples):
    """Format examples for code generation task"""
    texts = []

    for i in range(len(examples["instruction"])):
        instruction = examples["instruction"][i]
        code_output = examples["output"][i]

        # Qwen2.5-Coder chat template
        messages = [
            {"role": "system", "content": "You are Qwen-Coder, a helpful coding assistant. Generate clean, efficient, and well-commented code."},
            {"role": "user", "content": instruction},
            {"role": "assistant", "content": code_output}
        ]

        # Apply Qwen chat template
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False,
        )

        texts.append(text)

    return {"text": texts}

In [10]:
formatted_dataset = dataset.map(
    format_code_generation,
    batched=True,
    remove_columns=dataset.column_names,
    desc="Formatting for code generation"
)

Formatting for code generation:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [11]:
formatted_dataset

Dataset({
    features: ['text'],
    num_rows: 3000
})

In [13]:
print(formatted_dataset['text'][100])

<|im_start|>system
You are Qwen-Coder, a helpful coding assistant. Generate clean, efficient, and well-commented code.<|im_end|>
<|im_start|>user
Read a given list of strings and create a new list of all strings that are longer than 5 characters long.<|im_end|>
<|im_start|>assistant
def filter_long_strings(string_list):
 long_strings = []
 for string in string_list:
 if len(string) > 5:
 long_strings.append(string)
 return long_strings

string_list = ['Python', 'is', 'Fun']

long_strings = filter_long_strings(string_list)
print(long_strings)<|im_end|>



In [14]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=10,
    max_steps=150,  # More steps for code proficiency
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=5,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    seed=3407,
    output_dir="qwen_code_generator",
    save_strategy="no",
)

In [15]:
from trl import SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    args=training_args,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    packing=False,
)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [16]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,000 | Num Epochs = 1 | Total steps = 150
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 80,740,352 of 7,696,356,864 (1.05% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abeshith (abeshith-dr-m-g-r-educational-and-research-institute) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
5,1.668700
10,1.237800
15,0.910900
20,0.738100
25,0.574500
30,0.714900
35,0.454000
40,0.543500
45,0.638100
50,0.579200


TrainOutput(global_step=150, training_loss=0.649880649248759, metrics={'train_runtime': 1075.5711, 'train_samples_per_second': 1.116, 'train_steps_per_second': 0.139, 'total_flos': 1.2939212170579968e+16, 'train_loss': 0.649880649248759, 'epoch': 0.4})

In [17]:
model.save_pretrained("qwen_code_expert")
tokenizer.save_pretrained("qwen_code_expert")

('qwen_code_expert/tokenizer_config.json',
 'qwen_code_expert/special_tokens_map.json',
 'qwen_code_expert/chat_template.jinja',
 'qwen_code_expert/vocab.json',
 'qwen_code_expert/merges.txt',
 'qwen_code_expert/added_tokens.json',
 'qwen_code_expert/tokenizer.json')

In [ ]:
FastLanguageModel.for_inference(model)

In [19]:
def generate_code(task, language="Python", max_tokens=400):
    """Simple code generation function"""

    messages = [
        {"role": "system", "content": f"You are a helpful coding assistant. Generate clean, efficient {language} code with comments."},
        {"role": "user", "content": task}
    ]

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=0.3,  # Low temp for code accuracy
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    code = response.split("<|im_start|>assistant")[-1].replace("<|im_end|>", "").strip()

    return code

In [20]:
python_tasks = [
    "Write a Python function to implement bubble sort with time complexity comments",

    "Create a Python class for a binary search tree with insert, search, and delete methods",

    "Write a Python function to find the longest common subsequence between two strings using dynamic programming",

    "Implement a Python decorator that caches function results and measures execution time",

    "Create a Python function to merge two sorted linked lists",
]

print("🐍 PYTHON CODE GENERATION:")
for i, task in enumerate(python_tasks, 1):
    print(f"\n📝 Task {i}: {task}")
    code = generate_code(task, "Python", 350)
    print(f"Generated Code:\n{code}")
    print("-" * 50)

🐍 PYTHON CODE GENERATION:

📝 Task 1: Write a Python function to implement bubble sort with time complexity comments
Generated Code:
system
You are a helpful coding assistant. Generate clean, efficient Python code with comments.
user
Write a Python function to implement bubble sort with time complexity comments
assistant
def bubble_sort(arr):
    n = len(arr)
 
    # Traverse through all array elements
    for i in range(n-1):
        swapped = False
 
        # Last i elements are already in place
        for j in range(0, n-i-1):
 
            # traverse the array from 0 to n-i-1
            # Swap if the element found is greater than the next element
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]
                swapped = True
 
        # If no two elements were swapped by inner loop, then break
        if not swapped:
            break

# Time Complexity: O(n^2)
--------------------------------------------------

📝 Task 2: Create a Python class

In [21]:
js_tasks = [
    "Write a JavaScript function to implement quicksort algorithm",

    "Create a JavaScript class for handling HTTP requests with async/await",

    "Write a JavaScript function to debounce user input events",

    "Implement a JavaScript function to deep clone an object",
]

print("\n🟨 JAVASCRIPT CODE GENERATION:")
for i, task in enumerate(js_tasks, 1):
    print(f"\n📝 Task {i}: {task}")
    code = generate_code(task, "JavaScript", 300)
    print(f"Generated Code:\n{code}")
    print("-" * 50)


🟨 JAVASCRIPT CODE GENERATION:

📝 Task 1: Write a JavaScript function to implement quicksort algorithm
Generated Code:
system
You are a helpful coding assistant. Generate clean, efficient JavaScript code with comments.
user
Write a JavaScript function to implement quicksort algorithm
assistant
function quickSort(arr) {
  if (arr.length <= 1) return arr;

  let pivot = arr[Math.floor(arr.length / 2)];
  let left = [];
  let right = [];

  for (let i = 0; i < arr.length; i++) {
    if (i === Math.floor(arr.length / 2)) continue;
    if (arr[i] < pivot) {
      left.push(arr[i]);
    } else {
      right.push(arr[i]);
    }
  }

  return [...quickSort(left), pivot, ...quickSort(right)];
}
--------------------------------------------------

📝 Task 2: Create a JavaScript class for handling HTTP requests with async/await
Generated Code:
system
You are a helpful coding assistant. Generate clean, efficient JavaScript code with comments.
user
Create a JavaScript class for handling HTTP requests

In [22]:
algorithm_tasks = [
    "Implement Dijkstra's shortest path algorithm in Python",

    "Write a function to solve the N-Queens problem using backtracking",

    "Create a function to implement the Knapsack problem using dynamic programming",
]

print("\n🧮 ALGORITHM IMPLEMENTATION:")
for i, task in enumerate(algorithm_tasks, 1):
    print(f"\n📝 Task {i}: {task}")
    code = generate_code(task, "Python", 400)
    print(f"Generated Code:\n{code}")
    print("-" * 50)


🧮 ALGORITHM IMPLEMENTATION:

📝 Task 1: Implement Dijkstra's shortest path algorithm in Python
Generated Code:
system
You are a helpful coding assistant. Generate clean, efficient Python code with comments.
user
Implement Dijkstra's shortest path algorithm in Python
assistant
def dijkstra(graph, start):
    # Initialize distances to infinity and set the distance of the starting node to 0
    distances = {node: float('inf') for node in graph}
    distances[start] = 0

    # Set up a priority queue to store nodes to be visited
    pq = [(0, start)]

    while pq:
        current_distance, current_node = heapq.heappop(pq)

        # If we have already found a shorter path to this node, skip it
        if current_distance > distances[current_node]:
            continue

        # For each neighbor of the current node, calculate the distance from the start
        for neighbor, weight in graph[current_node].items():
            distance = current_distance + weight

            # If we find 

In [23]:
completion_tasks = [
    "Complete this Python function: def fibonacci_generator():",

    "Complete this JavaScript function: function validateEmail(email) {",

    "Complete this Python class: class LRUCache:",
]

print("\n🔄 CODE COMPLETION:")
for i, task in enumerate(completion_tasks, 1):
    print(f"\n📝 Task {i}: {task}")
    code = generate_code(task, max_tokens=250)
    print(f"Completed Code:\n{code}")
    print("-" * 50)


🔄 CODE COMPLETION:

📝 Task 1: Complete this Python function: def fibonacci_generator():
Completed Code:
system
You are a helpful coding assistant. Generate clean, efficient Python code with comments.
user
Complete this Python function: def fibonacci_generator():
assistant
def fibonacci_generator():
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b
--------------------------------------------------

📝 Task 2: Complete this JavaScript function: function validateEmail(email) {
Completed Code:
system
You are a helpful coding assistant. Generate clean, efficient Python code with comments.
user
Complete this JavaScript function: function validateEmail(email) {
assistant
function validateEmail(email) {
  var re = /^(([^<>()\[\]\\.,;:\s@"]+(\.[^<>()\[\]\\.,;:\s@"]+)*)|(".+"))@((\[[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}])|(([a-zA-Z\-0-9]+\.)+[a-zA-Z]{2,}))$/;
  return re.test(String(email).toLowerCase());
}
--------------------------------------------------

📝 Task

In [24]:
if torch.cuda.is_available():
    print(f"\n📊 GPU Memory Usage:")
    print(f"Allocated: {torch.cuda.memory_allocated()/1024**3:.2f} GB")
    print(f"Reserved: {torch.cuda.memory_reserved()/1024**3:.2f} GB")


📊 GPU Memory Usage:
Allocated: 5.90 GB
Reserved: 11.84 GB
